In [ ]:
!unzip twitter_bootcamp-covid.zip # Descompactação do arquivo com os JSON

In [ ]:
import os
path = '/content/twitter_bootcamp-covid'
path_list = os.listdir(path)
for i in range(len(path_list)):
  path_list[i] = f'{path}/{path_list[i]}'

# Criação do caminho de todos os JSOM
files_list = []
for i in path_list:
  for j in os.listdir(i):
    for k in os.listdir(f'{i}/{j}'):
      files_list.append(f'{i}/{j}/{k}') # Lista com o caminho de todos os JSON com os dados do twitter

In [ ]:
import pandas as pd
import json

json_list = []
for f in files_list:
  with open(f,'r') as fd:
    for line in fd:
      data = json.loads(line) # Cada linha precisa ser lida como um JSON independente.
      data['root_file_path'] = f # Essa chave será uma coluna no dataframe que indicará a origem do arquivo 
      json_list.append(data) # Cada JSON independente é anexado a uma lista

In [ ]:
df = pd.json_normalize(json_list) # A função de normalização do pandas permite estruturar os JSONs de forma tabular
df = df[['data', 'root_file_path']].explode('data').reset_index(drop = True)
df = pd.concat([df, pd.json_normalize(df['data'])], axis = 1).drop('data', axis = 1)
df.to_csv('twitter_all.csv') # E então salvar esta tabela em algum formato

# Tratamento dos dados brutos

In [ ]:
df = pd.read_csv('twitter_all.csv')
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,5,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,root_file_path,author_id,edit_history_tweet_ids,created_at,id,conversation_id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.place_id,in_reply_to_user_id,geo.coordinates.type,geo.coordinates.coordinates,withheld.copyright,withheld.country_codes,withheld.scope
0,0,/content/twitter_bootcamp-covid/ES/extract_dat...,1.219887e+09,['1355665810627440643'],2021-01-30T23:54:50.000Z,1355665810627440643,1.355666e+18,El covid son los padres. Se ríen en nuestra pu...,0.0,0.0,1.0,0.0,e00abb3527a00356,NaN,NaN,NaN,NaN,NaN,NaN
1,1,/content/twitter_bootcamp-covid/ES/extract_dat...,3.894440e+08,['1355664979127640067'],2021-01-30T23:51:32.000Z,1355664979127640067,1.355665e+18,Ahora hay Covid y el lunes tengo dos exámenes.,0.0,0.0,0.0,0.0,206c436ce43a43a3,389444004.0,NaN,NaN,NaN,NaN,NaN
2,2,/content/twitter_bootcamp-covid/ES/extract_dat...,4.667542e+08,['1355663971756482561'],2021-01-30T23:47:32.000Z,1355663971756482561,1.355664e+18,#L6Neneronegro Estais hablando de la Covid y l...,0.0,1.0,3.0,0.0,715325aceccc7851,NaN,NaN,NaN,NaN,NaN,NaN
3,3,/content/twitter_bootcamp-covid/ES/extract_dat...,7.673609e+08,['1355662553674895366'],2021-01-30T23:41:54.000Z,1355662553674895366,1.355661e+18,@Desahogada Q el covid era broma q mañana farr...,0.0,0.0,2.0,0.0,cd43ea85d651af92,9158592.0,NaN,NaN,NaN,NaN,NaN
4,4,/content/twitter_bootcamp-covid/ES/extract_dat...,1.087832e+18,['1355661597507727360'],2021-01-30T23:38:06.000Z,1355661597507727360,1.355661e+18,@Desahogada Q las vacunas del covid no eran ef...,1.0,0.0,3.0,0.0,731c9d11275a5436,9158592.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Como o objetivo é uma análise de sentimento, a maior parte das colunas são desnecessárias. Para nós, as únicas colunas importantes são: text, created_at, geo.place_id e root_file_path

df = df[['text', 'created_at', 'geo.place_id', 'root_file_path']]

# O texto é a feature mais importante, sendo assim, as linhas onde o texto é nulo serão descartadas

df.drop(df[df['text'].isna()].index, inplace = True)
df['created_at'] = pd.to_datetime(df['created_at'], format = "%Y-%m-%dT%H:%M:%S.%fZ")
df['root_file_path'] = df['root_file_path'].str.split('/')
df['root_file_path'] = df['root_file_path'].apply(lambda x: x[3])
df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text,created_at,geo.place_id,root_file_path
0,El covid son los padres. Se ríen en nuestra pu...,2021-01-30 23:54:50,e00abb3527a00356,ES
1,Ahora hay Covid y el lunes tengo dos exámenes.,2021-01-30 23:51:32,206c436ce43a43a3,ES
2,#L6Neneronegro Estais hablando de la Covid y l...,2021-01-30 23:47:32,715325aceccc7851,ES
3,@Desahogada Q el covid era broma q mañana farr...,2021-01-30 23:41:54,cd43ea85d651af92,ES
4,@Desahogada Q las vacunas del covid no eran ef...,2021-01-30 23:38:06,731c9d11275a5436,ES


In [ ]:
# Filtros para separação dos datasets de acordo com os países

df_ES = df[df['root_file_path'] == 'ES']
df_EC = df[df['root_file_path'] == 'EC']
df_AR = df[df['root_file_path'] == 'AR']
df_CL = df[df['root_file_path'] == 'CL']
df_BR = df[df['root_file_path'] == 'BR']
df_MX = df[df['root_file_path'] == 'MX']

In [ ]:
df.to_csv('/content/tratados/twitter_all_limpo.csv')
df_EC.to_csv('/content/tratados/df_EC.csv')
df_AR.to_csv('/content/tratados/df_AR.csv')
df_CL.to_csv('/content/tratados/df_CL.csv')
df_BR.to_csv('/content/tratados/df_BR.csv')
df_MX.to_csv('/content/tratados/df_MX.csv')
df_ES.to_csv('/content/tratados/df_ES.csv')

In [ ]:
!zip -r /content/download.zip /content/tratados # Compactação dos arquivos finais

  adding: content/tratados/ (stored 0%)
  adding: content/tratados/twitter_all_limpo.csv (deflated 59%)
  adding: content/tratados/df_ES.csv (deflated 57%)
  adding: content/tratados/df_EC.csv (deflated 61%)
  adding: content/tratados/df_AR.csv (deflated 59%)
  adding: content/tratados/df_BR.csv (deflated 59%)
  adding: content/tratados/df_MX.csv (deflated 58%)
  adding: content/tratados/df_CL.csv (deflated 59%)
